In [1]:
import os
import cv2
import torch
import numpy as np
from torchvision import transforms
from Line_Detection.model import LaneSegmentationModel

In [5]:
# ----- Config -----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = "checkpoints/new_best_val_loss.pt"

# ----- Load model -----
model = LaneSegmentationModel(num_classes=2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
checkpoint = torch.load(model_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)
model.eval()

LaneSegmentationModel(
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runni

In [6]:
# ----- Config -----
frames_folder = "data/CULane/video_example/05081544_0305"       # folder chứa ảnh từ video
output_path = "output_from_frames.mp4"
input_size = (512, 256)  # (width, height) khớp model

# ----- Frame list (sort theo thứ tự tên file) -----
frame_files = sorted([f for f in os.listdir(frames_folder) if f.endswith(('.jpg', '.png'))])
frame_paths = [os.path.join(frames_folder, f) for f in frame_files]

# ----- Init writer -----
first_frame = cv2.imread(frame_paths[0])
h, w = input_size[1], input_size[0]
fps = 30  # đặt FPS tùy ý hoặc từ metadata nếu có
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, input_size)

In [7]:
# ----- Transform -----
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((input_size[1], input_size[0])),  # Resize theo (H, W)
    transforms.ToTensor(),
])

In [18]:
# ----- Process frames -----
for frame_path in frame_paths:
    frame = cv2.imread(frame_path)
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    input_tensor = transform(img_rgb).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_tensor)
        pred = torch.argmax(output, dim=1).squeeze().cpu().numpy()

    # Convert mask to binary mask
    mask = (pred * 255).astype(np.uint8)
    mask = cv2.resize(mask, input_size)

    # Tạo mask màu đỏ (BGR)
    mask_color = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
    mask_color[mask == 255] = [0, 0, 255]  # Red

    # Overlay với ảnh gốc
    frame_resized = cv2.resize(frame, input_size)
    overlay = cv2.addWeighted(frame_resized, 0.7, mask_color, 0.3, 0)
    out.write(overlay)

# ----- Cleanup -----
out.release()
print(f"✅ Done! Video saved at: {output_path}")

✅ Done! Video saved at: output_from_frames.mp4
